## Pre-processing functions
- Differential rotation: *diff_rot*
- Cropping: *cropping*

In [1]:
def diff_rot(mapname, time_diff, basemap):
    
    #NOTE: to go back in time, time_diff MUST be negative
    in_time = mapname.date
    out_time = in_time + time_diff
    
    # The output frame is shifted in time for an observer at Earth (i.e., about five degrees offset in
    # heliographic longitude compared to the location of AIA in the original observation).
#     print(u.Quantity(mapname.meta['rsun_obs']), '\n"dsun_obs": ', u.Quantity(mapname.meta['dsun_obs'])) 
    out_frame = Helioprojective(observer='earth', obstime= out_time, rsun= mapname.meta['rsun_ref']*u.m)
    rot_frame = RotatedSunFrame(base=out_frame, rotated_time=in_time)
    
    # Construct a WCS object for the output map with the target RotatedSunHelioprojective frame specified
    # instead of the regular Helioprojective frame.
    out_shape = mapname.data.shape
    
    # WCS converts cdelt and cunit from arcsec to deg
    out_wcs = basemap.wcs
#     print('\nout_wcs:\n', out_wcs,'\nout_wcs type: ', type(out_wcs))
    out_wcs.coordinate_frame = rot_frame

#     For precise transformations of solar features, one should also use the context manager
#     transform_with_sun_center() to account for the translational motion of the Sun. Using the context manager,
#     the radius component stays as the solar radius as desired
#     Note: This context manager accounts only for the motion of the center of the Sun, i.e., translational motion.
#           The motion of solar features due to any rotation of the Sun about its rotational axis is not accounted for.
    # Reproject the map from the input frame to the output frame.
    with transform_with_sun_center():
        arr, _ = reproject_interp(mapname, out_wcs, out_shape)
    # Create the output map and preserve the original map’s plot settings.
    out_warp = sunpy.map.Map(arr, out_wcs)
    out_warp.plot_settings = mapname.plot_settings
    
    out_warp.meta["date-obs"]=mapname.meta["date-obs"]
#     print(out_warp.meta["date-obs"])
    out_warp.meta["instrume"]= mapname.meta['instrume']
    out_warp.meta["telescop"]= mapname.meta['telescop']
    out_warp.meta["wavelnth"] = mapname.meta['wavelnth']
    out_warp.meta["waveunit"] = mapname.meta["waveunit"]
    out_warp.meta["exptime"] = mapname.meta['exptime']
    
    out_warp.meta["cdelt1"]=mapname.meta['cdelt1']
    out_warp.meta["cdelt2"]=mapname.meta['cdelt2']
    out_warp.meta["cunit1"]=mapname.meta['cunit1']
    out_warp.meta["cunit2"]=mapname.meta['cunit2']
    out_warp.meta["rsun_obs"]= mapname.meta['rsun_obs']
    
    return out_warp



def cropping(mymap, top, bottom, vmin, vmax, plot=False):
    # cropping of the Map to specific coordinates
    #NOTE: the unit of measure of the cropping coordinates is degree
    top_right = SkyCoord(top[0] * u.arcsec, top[1] * u.arcsec, frame=mymap.coordinate_frame)
    bottom_left = SkyCoord(bottom[0] * u.arcsec, bottom[1] * u.arcsec, frame=mymap.coordinate_frame)

    swap_submap = mymap.submap(bottom_left, top_right=top_right)
    # Eliminate the presence of np.nan from the map (i.e., off-limb pixels)
    swap_submap_no_nan = swap_submap.data
    swap_submap_no_nan=swap_submap_no_nan[~np.isnan(swap_submap_no_nan)]
    print('Cropped Image')
    print('Mean value: ', np.mean(swap_submap_no_nan),'\n','Max value: ', np.max(swap_submap_no_nan),
         '\n', 'Min value: ', np.min(swap_submap_no_nan) )
    #print('Coordinate frame: \n', swap_submap.coordinate_frame)
    
    if plot==True:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection=swap_submap)
    
        swap_submap.plot(vmin=vmin, vmax=vmax)
        swap_submap.draw_limb()
        #swap_submap.draw_grid()
        plt.colorbar()
        plt.show()
    
    return swap_submap

## Region growing functions

#### Seeds research
- Two thresholds method: *find_seeds_2T*
- BD images: *find_seeds_BD*
- LBR images: *find_seeds_LBR*

#### Segmentation process
- neighbouring pixels research: *explore_neighbours*
- neighbouring pixels acquisition: *get_nearest_neighbour*
- segmenting function: *segment*

In [2]:
# Find seeds by defining a 2 Thresholding values (upper and lower one)
# default value: pixel ~30% less bright than the darkest pixel
def find_seeds_2T(data_img, min_data, thresh=[0.7, 1]):
    #Create an array containing the coordinates of the seed pixels, i.e. the ones
    #whose intensity is 30% less than the minimum pixel
    seeds=[]
    img_shape= data_img.shape
    for i in range(img_shape[0]): # row
        for k in range(img_shape[1]): #column
            if data_img[i][k]<=thresh[0]*min_data and data_img[i][k]>=thresh[1]*min_data:
                seeds.append([i,k])

    seeds=np.array(seeds)
    return(seeds)

#################################################################################
# Find seeds in a Base Difference map
def find_seeds_BD(data_img, thresh=-1, perc=0.3):
    # Threshold BD data by -1 DN (default value)
    # Extract 30% of the darkest pixels from it
    # OUTPUT: list of seeds
    
    min_data=np.min(data_img)
    seeds_list=[]
    image_to_sort_list= []
    img_shape= data_img.shape
    # Thresholding
    for i in range(img_shape[0]): # row
        for k in range(img_shape[1]): #column
            if data_img[i][k]<=thresh:
                image_to_sort_list.append(data_img[i][k])
                seeds_list.append([i,k])
    #print('Length of seeds list: ', len(seeds_list))
    seeds=np.array(seeds_list)
    image_to_sort=np.array(image_to_sort_list)
    ind_sort= np.argsort(image_to_sort)
    # Dark pixels definition and sorting in ascending order
    sorted_seeds=seeds[ind_sort]
    #print(len(sorted_seeds))
    #print('element 0: ', sorted_seeds[0], '\nvalue: ', data_img[sorted_seeds[0][0]][sorted_seeds[0][1]])
    
    # Extraction of the indicated "perc" value (default: 0.3) of the darkest pixels
    percent= round(perc*len(sorted_seeds))
    extracted_seeds=sorted_seeds[0:percent]
    #print('Dimension of extracted seeds: ', len(extracted_seeds))
    
    return(extracted_seeds)

#################################################################################
# Find seeds in a Logarithmic Base Ratio map
def find_seeds_LBR(data_img, thresh=-0.19, perc=0.3):
    # Threshold LBR data by -0.19 DN (default value)
    # Extract 30% (default value: perc) of the darkest pixels from it
    # OUTPUT: list of seeds
    
    min_data=np.min(data_img)
    seeds_list=[]
    image_to_sort_list= []
    img_shape= data_img.shape
    # Thresholding
    for i in range(img_shape[0]): # row
        for k in range(img_shape[1]): #column
            if data_img[i][k]<=thresh:
                image_to_sort_list.append(data_img[i][k])
                seeds_list.append([i,k])
    #print('Length of seeds list: ', len(seeds_list))
    seeds=np.array(seeds_list)
    image_to_sort=np.array(image_to_sort_list)
    ind_sort= np.argsort(image_to_sort)
    # Dark pixels definition and sorting in ascending order
    sorted_seeds=seeds[ind_sort]
    #print(len(sorted_seeds))
    #print('element 0: ', sorted_seeds[0], '\nvalue: ', data_img[sorted_seeds[0][0]][sorted_seeds[0][1]])
    
    # Extraction of the indicated "perc" value (default: 0.3) of the darkest pixels
    percent= round(perc*len(sorted_seeds))
    extracted_seeds=sorted_seeds[0:percent]
    #print('Dimension of extracted seeds: ', len(extracted_seeds))
    
    return(extracted_seeds)

##############################################################################################
# check if the neighbourhood pixels are inside the image and add them to the contour
def explore_neighbours(image_data, segmented_img, contour, current_pixel, img_shape, orientations):
    for orientation in orientations:
        neighbour = (current_pixel[0]+orientation[0], current_pixel[1]+orientation[1])

        # check if pixel is inside the image
        if 0<=neighbour[0]<(img_shape[0]) and 0<=neighbour[1]<(img_shape[1]):
            # create contour
            if segmented_img[neighbour[0],neighbour[1]]==0:
                contour.append(neighbour)
                segmented_img[neighbour[0],neighbour[1]]=150
        else:
            neighbour= None
            
    return contour

###############################################################################################
# establish which is the pixel, among the contour values, whose value is the closest to the seed value
def get_nearest_neighbour(image_data, contour, mean_seg_value):
    # Distance between contour data and data at seed coordinates (absolute value)
    dist_list = [abs(image_data[pixel[0], pixel[1]] - mean_seg_value) for pixel in contour]
    #print([image_data[pixel[0], pixel[1]]  for pixel in contour], '          ',
    #      [image_data[pixel[0], pixel[1]] - mean_seg_value  for pixel in contour])
    #print('Distance list: ', dist_list)
    #print('size of the difference array: ', len(dist_list))
    if len(dist_list)==0:
        #List of distances is empty
        return -1, 1000
    min_dist = min(dist_list)
    index = dist_list.index(min_dist)
    #print('minimum distance ', min_dist, 'minimum index ', index)
    
    return index, min_dist

In [2]:
###################################################################################
#                REGION GROWING SEGMENTATION FUNCTION

def segment(data_img, seed_points, threshold = -0.2):
    """
    Segment the image with the provided user seeds and thresholding value using region growing
    # data_img: numpy array of the LBR image to be segmented
    # seed_points: np.array #[row, column]
    # threshold: 
    
    """
    
    orientations = [(1,0),(0,1),(-1,0),(0,-1)] # 4 connectiviy points
    #orientations = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)] # 8 connectivity points
    img_shape= data_img.shape
    segmented_img = np.zeros(img_shape)
    
    for seed in seed_points:
        
        curr_pixel = [seed[0],seed[1]] #[row, column]
        if segmented_img[curr_pixel[0], curr_pixel[1]]==255: continue # pixel already explored
        
        contour = []
        seg_size = 1
        seg_value = (data_img[curr_pixel[0],curr_pixel[1]])
        #print(mean_seg_value)

        while(seg_value<threshold):
            # Include current pixel in segmentation
            segmented_img[curr_pixel[0], curr_pixel[1]]=255
            
            # Explore neighbours of current pixel
            contour = explore_neighbours(segmented_img, segmented_img, contour, curr_pixel, img_shape, orientations)
            #print('Contour pixels: ',contour)
            # Get the nearest neighbour
            nearest_neighbour_idx, dist = get_nearest_neighbour(data_img, contour, seg_value)
            
            # If no more neighbours to grow, move to the next seed
            if nearest_neighbour_idx==-1: break
            
            # Update Current pixel to the nearest neighbour
            curr_pixel = contour[nearest_neighbour_idx]
            
            # in case the new mean_seg_value is above the threshold, try to find another current pixel value
            # move in all the directions of the map, without stopping at the first non-valid value
            while (data_img[curr_pixel[0],curr_pixel[1]]>threshold): 
                # delete from contour once the current pixel does not respect the thresholding condition
                del contour[nearest_neighbour_idx]
                # find another nearest neighbour
                nearest_neighbour_idx, dist = get_nearest_neighbour(data_img, contour, seg_value)
                # if no more neighbours, move outside the loop
                if nearest_neighbour_idx==-1: break
                curr_pixel = contour[nearest_neighbour_idx]
            # If no more neighbours to grow, move to the next seed
            if nearest_neighbour_idx==-1: break
            
            # Increase segmentation size
            seg_size+=1
            # Update Mean pixel value for segmentation
            seg_value = data_img[curr_pixel[0],curr_pixel[1]]
            #print(seg_value)
            # Delete from contour once the nearest neighbour was chosen as the current node for expansion
            del contour[nearest_neighbour_idx]
     
    final_segm= segmented_img
    ## remove borders' value, because they are >-0.19DN (do not respect initial condition)
    final_segm[segmented_img==150]=0 
    return final_segm

## Area Analysis

In [ ]:
#############################################################################
# Find radiuses of circle at the Sun parallels
# rad - radius in pixels 
def  rad_parallel_cut (rad):
    out=np.zeros(2*rad+1)
    out = out.T
    for it in range (0,rad+1):
        out[it]=int(np.sqrt((rad**2-(rad-it)**2)))  
    #for it in range(rad+1, 2*rad+1):    
    out[rad+1:2*rad+1] = np.flipud(out[0:rad])
    #out = np.hstack((out,out[::-1]))
    return out



## Creating masks

In [ ]:
#############################################################################
## Where the segmented image is different from 255 (non-useful pixel), the BD image corresponding pixel shall be NaN
def masking_nan(map_img, segmented_img):
    mask= map_img.data
    mask[segmented_img.data<250]=np.nan # to mask
    masked_img=sunpy.map.Map(mask, map_img.meta)
    return masked_img

##############################################################################
##                         CREATING FULL DISK MASK
# Starting from the BD or LBR full-disk image, the function creates a full-disk mask with the same size as the
# BD or LBR map. The mask does not cover the dimming area, as it is based upon the segmented image.

def full_disk_mask(BD_full_img, segmented_img):
    # BD_full_img : full-disk processed image
    # segmented_img : corresponding cropped segmented image
    
    final_size = BD_full_img.data.shape # [rows, columns]
    crpix_x_full = int(BD_full_img.meta['crpix1'])
    crpix_y_full = int(BD_full_img.meta['crpix2'])
    crpix_x_crop = int(segmented_img.meta['crpix1'])
    crpix_y_crop = int(segmented_img.meta['crpix2'])
    # coordinate of the bottom left pixel
    xpix= crpix_x_full - crpix_x_crop
    ypix= crpix_y_full - crpix_y_crop
#     print(xpix, ypix)
    
    # Create final mask
    data = segmented_img.data
    height, length= data.shape
#     print('Height: ', height,'Length: ', length)
    fulldisk_data= np.zeros(final_size)
    
#     ## Using PIL library
#     image_full=Image.fromarray(fulldisk_mask)
#     image_crop=Image.fromarray(data)    
#     # box: a 2-tuple is used instead, it’s treated as the upper left corner: (xpix, ypix).
#     # Note: Image coordinate system starts from the upper left corner
#     box= (xpix, ypix)
#     Image.Image.paste(image_full, image_crop, box=box)
#     fulldisk_data=np.array(image_full)
    
    ## Using NUMPY
    for row in range(height):
        fulldisk_data[ypix+row, xpix:(xpix+length)]=data[row]
    
    fulldisk_mask= sunpy.map.Map(fulldisk_data, BD_full_img.meta)
#     fulldisk_mask.plot()
#     fulldisk_mask.draw_limb()
    return fulldisk_mask

## Plotting

In [ ]:
##############################################################################################
# Define colorbar for a map

def colorbar(limits, span, label= None):
    # limits= [vmin, vmax] : set lower and upper limit for the colorbar
    # span (integer number) : interval between colorbar values (Note: from lower to upper value, with step defined by "span")
    # label (string): title of the colorbar
    vmin=limits[0]
    vmax=limits[1]
    cbar= plt.colorbar()

    cbar.locator = matplotlib.ticker.FixedLocator(range(vmin,vmax+1, span))
    if label != None:
        cbar.set_label(label, labelpad=2, rotation=90)
    cbar.update_ticks()
    cbar.ax.xaxis.set_ticks_position('bottom')
    
#############################################################################
## Plot a rectangle starting from the box coordinates 
# box [x coord, y coord]: coordinates of the bottom left pixel
def make_pixel_rect(box, size, axes, color='red'):
    rect= Rectangle(box, size, size, color=color)
    axes.add_patch(rect)
    return

#############################################################################
## Plot a surrounding rectangle starting from the box coordinates, with specific height and width and title
# xy=(x_coord, y_coord)
def surrounding_rect(xy, width, height, axes, name, color='red', linewidth=0.5, position='top'):
    rect= Rectangle(xy, width, height, color=color, fill=False, linewidth=linewidth)
    axes.add_patch(rect)
    rx, ry = rect.get_xy()
    cx = rx + rect.get_width()/2
    if position=='top':
        i=1
        k=1
    elif position=='bottom':
        i=-1
        k=0
    else:
        raise ValueError
    cy = ry + (rect.get_height()*k + 10)*i
    axes.annotate(name, (cx, cy), color='red', fontsize=6, ha='center', va='center')
    return

## Pixel boxes analysis

In [ ]:

#############################################################################
##                      MEAN VALUE OF INTENSITY
# 

def dimming_avg_intensity(coord, smap, seg_map, size=3):
    ## Note: if the pixel is not a dimming pixel, then its value is NaN 
    ## The intensity value is given by the mean value of the non-NaN pixels
    # coord = [ x coord, y coord] : coordinate of the bottom left pixel of the box
    # smap : map where the points of interest are
    # size : size of the square box (default: 3x3 box)
    
    col= coord[0] # x coordinate
    row= coord[1] # y coordinate
    size = 3
    
    # Only dimming pixel must be available
    new_map= masking_nan(smap, seg_map)
#     print('New map coordinates:\n',new_map.data[row:row+size, col:col+size])
#     plt.figure()
#     new_map.plot()
#     plt.colorbar()
    # Compute intensity of the box
#     intensity = np.mean(new_map.data[row:row+size, col:col+size])
    pix= new_map.data[row:row+size, col:col+size]
    valid= pix[pix==pix]
    if len(valid>0):
        intensity= np.mean(valid)
    else:
        intensity= np.nan

    return intensity

###############################################################################################
##

def box_avg_intensity(coord, smap, seg_map, size=3):
    ## Note: if the pixel is not a dimming pixel, then its value is NaN and so is the value of the box
    # coord= [ x coord, y coord] : coordinate of the bottom left pixel
    # smap : map where the points of interest are
    # size : size of the square box (default: 3x3 box)
    
    col= coord[0] # x coordinate
    row= coord[1] # y coordinate
    size = 3
    
    # Compute intensity of the box
    intensity = np.mean(smap.data[row:row+size, col:col+size])
    
    return intensity


#############################################################################
##                        SUM OF INTENSITY

def dimming_tot_intensity(coord, smap, seg_map, size=3):
    ## Note: if the pixel is not a dimming pixel, then its value is NaN 
    ## The intensity value is given by the sum of the non-NaN pixels
    
    # coord= [ x coord, y coord] : coordinate of the bottom left pixel
    # smap : map where the points of interest are
    # size : size of the square box (default: 3x3 box)
    
    col= coord[0] # x coordinate
    row= coord[1] # y coordinate
    size = 3
    
    # Only dimming pixel must be available
    new_map= masking_nan(smap, seg_map)
    pix= new_map.data[row:row+size, col:col+size]
    valid= pix[pix==pix]
    if len(valid>0):
        intensity= sum(valid)
    else:
        intensity= np.nan

    return intensity

###############################################################################################
##

def box_tot_intensity(coord, smap, seg_map, size=3):
    ## Note: if the pixel is not a dimming pixel, then its value is NaN and so is the value of the box
    # coord= [ x coord, y coord] : coordinate of the bottom left pixel
    # smap : map where the points of interest are
    # size : size of the square box (default: 3x3 box)
    
    col= coord[0] # x coordinate
    row= coord[1] # y coordinate
    size = 3
    
    # Compute intensity of the box
    box_array = smap.data[row:row+size, col:col+size]
    intensity = np.sum(box_array)
    
    return intensity